In [ ]:
# @title 連接google drive
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi
import os
os.chdir('/content/drive/My Drive')
os.getcwd()
os.chdir('/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data')
os.getcwd()

Mounted at /content/drive
/bin/bash: line 1: nvidia-smi: command not found


'/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
!pip install networkx polyline

import polyline
import re
import itertools
import requests

In [ ]:
# 读取数据框
# df = pd.read_csv('1_0_df.csv')
# dft = pd.read_csv('t1_0_time_diffs.csv')
df_folder_path = '/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data'  # 替换为实际的路径
dft_folder_path = '/content/drive/My Drive/碳數據科技黑客松/t'  # 替换为实际的路径

# 初始化 DataFrame 列表
df_list = []
dft_list = []


# 转换时间数据
dfo = pd.read_csv('/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/fi5.csv')
# 获取 date_category 列的唯一值
date_categories = dfo['date_category'].unique()
# 将数组顺序倒过来
date_categories = date_categories[::-1]


location_clusters = dfo['location_cluster'].unique()
location_cols = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']

def get_route_distance(lat1, lon1, lat2, lon2, distance_cache):
    # 构建请求的键
    cache_key = (lat1, lon1, lat2, lon2)

    # 检查缓存中是否已有该路径的距离
    if cache_key in distance_cache:
        return distance_cache[cache_key]

    # 如果缓存中没有，发起新的 API 请求
    url = f"http://router.project-osrm.org/route/v1/car/{lon1},{lat1};{lon2},{lat2}?overview=false"
    try:
        response = requests.get(url)
        response.raise_for_status()  # 检查是否有 HTTP 错误
        data = response.json()

        if 'routes' in data and len(data['routes']) > 0:
            distance = data['routes'][0]['distance'] / 1000
            # 存储距离到缓存
            distance_cache[cache_key] = distance
            return distance
        else:
            return None
    except requests.RequestException as e:
        print(f"Request Error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON Decoding Error: {e}")
        return None


def process_dataframe(df, distance_cache):
    # 提取每行的餐厅和送餐点位置
    points = df[location_cols].values.tolist()

    # 分别提取所有餐厅和所有送餐点的坐标
    restaurants = [row[:2] for row in points]
    deliveries = [row[2:] for row in points]

    # 初始化最短路径和最小距离
    shortest_path = None
    min_distance = float('inf')

    # 使用 tqdm 来显示进度条
    for rest_order in itertools.permutations(restaurants):
        for del_order in itertools.permutations(deliveries):
            total_distance = 0
            # 计算所有餐厅之间的距离
            for i in range(len(rest_order) - 1):
                distance = get_route_distance(rest_order[i][0], rest_order[i][1], rest_order[i + 1][0], rest_order[i + 1][1],distance_cache)
                if distance is None:
                    total_distance = float('inf')
                    break
                total_distance += distance

            # 计算最后一个餐厅到第一个送餐点的距离
            if total_distance != float('inf'):
                distance = get_route_distance(rest_order[-1][0], rest_order[-1][1], del_order[0][0], del_order[0][1],distance_cache)
                if distance is None:
                    continue
                total_distance += distance

            # 计算所有送餐点之间的距离
            for i in range(len(del_order) - 1):
                distance = get_route_distance(del_order[i][0], del_order[i][1], del_order[i + 1][0], del_order[i + 1][1],distance_cache)
                if distance is None:
                    total_distance = float('inf')
                    break
                total_distance += distance

            if total_distance < min_distance:
                min_distance = total_distance
                shortest_path = list(rest_order) + list(del_order)

    return shortest_path, min_distance

def convert_time(df, columns):
    for col in tqdm(columns, desc="Converting Time"):
        # 创建新列以存储转换后的时间数据
        df[col + '_in_minutes'] = pd.to_datetime(df[col], format='%H:%M:%S').dt.hour * 60 + pd.to_datetime(df[col], format='%H:%M:%S').dt.minute
    return df
# 转换时间列
time_cols = ['Time_Orderd']
dfo = convert_time(dfo, time_cols)



# # # 检查之前的结果文件是否存在
results_file = '/content/drive/My Drive/碳數據科技黑客松/final_group_results_detailed.csv'
if os.path.exists(results_file):
    # 如果存在，读取之前的结果
    all_final_results_df = pd.read_csv(results_file)
else:
    # 否则，初始化一个空的 DataFrame
    all_final_results_df = pd.DataFrame()
# all_final_results_df = pd.DataFrame()

for date_category in date_categories:
    for location_cluster in location_clusters:
        # 构建文件名
        df_file_name = f"{date_category}_{location_cluster}_df.csv"
        dft_file_name = f"t{date_category}_{location_cluster}_time_diffs.csv"

        # 构建完整的文件路径
        df_file_path = os.path.join(df_folder_path, df_file_name)
        dft_file_path = os.path.join(dft_folder_path, dft_file_name)
        print(df_file_path,dft_file_path)

        single_group_file = f"/content/drive/My Drive/碳數據科技黑客松/singledf/{date_category}_{location_cluster}_single_group.csv"

        # 检查是否已经处理过该组
        if os.path.exists(single_group_file):
            print(f"Group {date_category}_{location_cluster} already processed. Skipping...")
            continue
         # 检查文件是否存在且非空
        if os.path.exists(df_file_path) and os.path.getsize(df_file_path) > 0 and \
           os.path.exists(dft_file_path) and os.path.getsize(dft_file_path) > 0:
            print(f"start: group{date_category}_{location_cluster}")
            try:
                # 尝试读取文件
                df = pd.read_csv(df_file_path)
                dft = pd.read_csv(dft_file_path)
                # 处理数据，代码不变...
            except pd.errors.EmptyDataError:
                # 捕获空数据错误
                print(f"CSV file {df_file_name} or {dft_file_name} is empty or invalid. Skipping...")
                continue  # 跳过当前迭代，继续下一个循环
        # # 检查文件是否存在且非空
        # if os.path.exists(df_file_path) and os.path.getsize(df_file_path) > 0 and \
        #    os.path.exists(dft_file_path) and os.path.getsize(dft_file_path) > 0:
        #     print(f"start: group{date_category}_{location_cluster}")

        #     df = pd.read_csv(df_file_path)
        #     dft = pd.read_csv(dft_file_path)

#             # 添加到列表
#             df_list.append(df)
#             dft_list.append(dft)


            # 获取所有的独特索引
            unique_indices = set()
            for col in df.columns:
                match = re.match(r"\((\d+), 'restaurant'\)", col)
                if match:
                    unique_indices.add(int(match.group(1)))


            # 初始化一个空的列表来存储每行的数据
            rows_to_add = []

            for _, row in df.iterrows():
                # 使用original_index作为每行的唯一标识符
                row_data = {'original_index': row['original_index']}
                distances = []

                for index in unique_indices:
                    # 构造 df 中的列名
                    restaurant_col = f"({index}, 'restaurant')"
                    delivery_col = f"({index}, 'delivery')"

                    # 构造 dft 中的列名
                    dft_col = str(index)

                    # 检查列是否存在于两个数据框中
                    if restaurant_col in df.columns and delivery_col in df.columns:
                        # 将 df 中的两列相加
                        total_distance = row[restaurant_col] + row[delivery_col]
                        # 存储距离和索引
                        distances.append((index, total_distance))

                # 筛选 dft 中20分钟以内的索引
                close_time_indices = dft.columns[(dft.loc[_, :] <= 20).values].tolist()

                # 筛选出在 dft 中20分钟以内的 distances 元素
                distances_within_20_min = [(idx, dist) for idx, dist in distances if str(idx) in close_time_indices]

                # 确保 distances_within_20_min 不为空
                if distances_within_20_min:
                    # 对 distances_within_20_min 进行排序
                    sorted_distances = sorted(distances_within_20_min, key=lambda x: x[1])

                    # 取前10个距离的
                    top_10_indices = [idx for idx, _ in sorted_distances[:5]]
                else:
                    top_10_indices = []

                # 将排名前10的索引添加到row_data中
                row_data['top_10_indices'] = top_10_indices

                # 将row_data添加到rows_to_add列表
                rows_to_add.append(row_data)

            # 使用pandas.concat来添加所有行到combined_results
            combined_results = pd.DataFrame(rows_to_add)

            combined_results['top_10_indices'] = combined_results['top_10_indices'].apply(lambda lst: [int(x) for x in lst])
            # 将 'original_index' 转换为整数
            combined_results['original_index'] = combined_results['original_index'].astype(int)

            # 将DataFrame转换为字典格式以方便处理
            top_10_dict = combined_results.set_index('original_index')['top_10_indices'].to_dict()

            # 初始化用于存储组的列表
            groups = []

            # 创建组
            for index, indices in top_10_dict.items():
                potential_group = set(indices)  # 创建一个潜在的组
                potential_group.add(index)  # 加上自身

                # 逐步筛选掉不符合条件的成员
                for member in list(potential_group):
                    if not all(member in top_10_dict.get(other, []) for other in potential_group if other != member):
                        potential_group.discard(member)

                # 确保每个组至少有两个成员
                if len(potential_group) >= 2 and potential_group not in groups:
                    groups.append(potential_group)

            # # 输出找到的组
            # for group in groups:
            #     print(group)


            distance_cache={}

            # 初始化用于存储每个数据点最佳组的字典
            best_group_for_points = {}

            # 初始化用于存储每组的碳减排量
            group_carbon_reductions = {}

            # 遍历每个组
            for group in groups:
                group_list = list(group)
                group_subset = dfo.loc[group_list]
                total_distance_km = group_subset['Distance_km'].sum()
                min_distance = process_dataframe(group_subset, distance_cache)[1]
                carbon_reduction_g = (total_distance_km - min_distance) * 50.8

                # 如果 carbon_reduction_g 为负，则跳过当前组
                if carbon_reduction_g < 0:
                    continue

                # 存储每个组的碳减排量
                group_carbon_reductions[tuple(group_list)] = carbon_reduction_g

            # 为每个点找到最佳组
            for point in dfo.index:
                best_group = None
                best_carbon_reduction = -float('inf')

                for group, carbon_reduction in group_carbon_reductions.items():
                    if point in group and carbon_reduction > best_carbon_reduction:
                        best_carbon_reduction = carbon_reduction
                        best_group = group

                if best_group is not None:
                    best_group_for_points[point] = best_group
            # 根据最佳组创建最终结果
            final_results = []

            for point, best_group in best_group_for_points.items():
                group_subset = dfo.loc[list(best_group)]
                max_time_diff_minutes = group_subset['Time_Orderd_in_minutes'].max() - group_subset['Time_Orderd_in_minutes'].min()
                carbon_reduction_g = group_carbon_reductions[best_group]

                                # 计算时间差和最短路径距离
                max_time_diff_minutes = group_subset['Time_Orderd_in_minutes'].max() - group_subset['Time_Orderd_in_minutes'].min()
                min_order_time = group_subset['Time_Orderd_in_minutes'].min()

                min_distance = process_dataframe(group_subset, distance_cache)[1]
                total_distance_km = group_subset['Distance_km'].sum()

                            # 对 group_subset 的每一行进行迭代
                for idx, row in group_subset.iterrows():
                    row = dfo.loc[idx]
                    waiting_time = row['Time_Orderd_in_minutes'] - min_order_time  # 计算等待时间

                    # 创建一个包含额外信息的字典
                    row_dict = row.to_dict()
                    row_dict.update({
                        'Group': group,
                        'Idx': idx,
                        'Restaurant_Lat': row['Restaurant_latitude'],
                        'Restaurant_Lon': row['Restaurant_longitude'],
                        'Delivery_Lat': row['Delivery_location_latitude'],
                        'Delivery_Lon': row['Delivery_location_longitude'],
                        'Total_Distance_km': total_distance_km,
                        'Max_Time_Diff_Minutes': max_time_diff_minutes,
                        'Shortest_Path_Distance_km': min_distance,
                        'Carbon_Reduction_g': carbon_reduction_g,
                        'Group_Length': len(group_subset),
                        'Waiting_Time_Minutes': waiting_time
                    })

                    # 添加到结果列表
                    final_results.append(row_dict)


            # 创建最终 DataFrame
            final_results_df = pd.DataFrame(final_results)


            # final_results_df.to_csv('final_group_results.csv', index=False)

          # 创建单个组的 DataFrame
            single_group_df = pd.DataFrame(final_results)

            # 保存单个组的 DataFrame
            single_group_df.to_csv(f"/content/drive/My Drive/碳數據科技黑客松//singledf/{date_category}_{location_cluster}_single_group.csv", index=False)

            # 将单个组的 DataFrame 添加到全部结果的列表中
            all_final_results_df = pd.concat([all_final_results_df, single_group_df], ignore_index=True)
            all_final_results_df.to_csv(results_file, index=False)


# 打印或保存 DataFrame
all_final_results_df.to_csv('/content/drive/My Drive/碳數據科技黑客松/final_group_results_detailed.csv', index=False)


Converting Time: 100%|██████████| 1/1 [00:00<00:00, 29.25it/s]


/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/43_1_df.csv /content/drive/My Drive/碳數據科技黑客松/t/t43_1_time_diffs.csv
Group 43_1 already processed. Skipping...
/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/43_2_df.csv /content/drive/My Drive/碳數據科技黑客松/t/t43_2_time_diffs.csv
Group 43_2 already processed. Skipping...
/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/43_10_df.csv /content/drive/My Drive/碳數據科技黑客松/t/t43_10_time_diffs.csv
Group 43_10 already processed. Skipping...
/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/43_8_df.csv /content/drive/My Drive/碳數據科技黑客松/t/t43_8_time_diffs.csv
Group 43_8 already processed. Skipping...
/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/43_9_df.csv /content/drive/My Drive/碳數據科技黑客松/t/t43_9_time_diffs.csv
Group 43_9 already processed. Skipping...
/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/43_5_df.csv /content/drive/My Drive/碳數據科技黑客松/t/t43_5_time_diffs.csv
Group 43_5 already processed. Skipping...
/content/driv

NameError: ignored

In [ ]:
import zipfile

def unzip_file(zip_path, extract_to):
    """
    Unzip a zip file to the specified directory.

    :param zip_path: Path to the zip file
    :param extract_to: Directory where files should be extracted
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Files extracted to {extract_to}")

# 使用範例
zip_file_path = '/content/drive/My Drive/112 Sesmester/data2/filter_data-20231207T060751Z-001.zip'  # 更改為您的 ZIP 檔案路徑
extract_to_path1 = '/content/drive/My Drive/112 Sesmester/data2/location'  # 更改為您想要解壓縮到的目錄路徑

unzip_file(zip_file_path, extract_to_path1)

Files extracted to /content/drive/My Drive/112 Sesmester/data2/location


In [ ]:



location_cols = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']

def convert_time(df, columns):
    for col in tqdm(columns, desc="Converting Time"):
        # 创建新列以存储转换后的时间数据
        df[col + '_in_minutes'] = pd.to_datetime(df[col], format='%H:%M:%S').dt.hour * 60 + pd.to_datetime(df[col], format='%H:%M:%S').dt.minute
    return df
# 转换时间列
time_cols = ['Time_Orderd']

# 获取所有的独特索引
unique_indices = set()
for col in df.columns:
    match = re.match(r"\((\d+), 'restaurant'\)", col)
    if match:
        unique_indices.add(int(match.group(1)))


# 初始化一个空的列表来存储每行的数据
rows_to_add = []

for _, row in df.iterrows():
    # 使用original_index作为每行的唯一标识符
    row_data = {'original_index': row['original_index']}
    distances = []

    for index in unique_indices:
        # 构造 df 中的列名
        restaurant_col = f"({index}, 'restaurant')"
        delivery_col = f"({index}, 'delivery')"

        # 构造 dft 中的列名
        dft_col = str(index)

        # 检查列是否存在于两个数据框中
        if restaurant_col in df.columns and delivery_col in df.columns:
            # 将 df 中的两列相加
            total_distance = row[restaurant_col] + row[delivery_col]
            # 存储距离和索引
            distances.append((index, total_distance))

    # 筛选 dft 中20分钟以内的索引
    close_time_indices = dft.columns[(dft.loc[_, :] <= 20).values].tolist()

    # 筛选出在 dft 中20分钟以内的 distances 元素
    distances_within_20_min = [(idx, dist) for idx, dist in distances if str(idx) in close_time_indices]

    # 确保 distances_within_20_min 不为空
    if distances_within_20_min:
        # 对 distances_within_20_min 进行排序
        sorted_distances = sorted(distances_within_20_min, key=lambda x: x[1])

        # 取前10个距离的索引
        top_10_indices = [idx for idx, _ in sorted_distances[:5]]
    else:
        top_10_indices = []

    # 将排名前10的索引添加到row_data中
    row_data['top_10_indices'] = top_10_indices

    # 将row_data添加到rows_to_add列表
    rows_to_add.append(row_data)

# 使用pandas.concat来添加所有行到combined_results
combined_results = pd.DataFrame(rows_to_add)

combined_results['top_10_indices'] = combined_results['top_10_indices'].apply(lambda lst: [int(x) for x in lst])
# 将 'original_index' 转换为整数
combined_results['original_index'] = combined_results['original_index'].astype(int)

# 将DataFrame转换为字典格式以方便处理
top_10_dict = combined_results.set_index('original_index')['top_10_indices'].to_dict()

# 初始化用于存储组的列表
groups = []

# 创建组
for index, indices in top_10_dict.items():
    potential_group = set(indices)  # 创建一个潜在的组
    potential_group.add(index)  # 加上自身

    # 逐步筛选掉不符合条件的成员
    for member in list(potential_group):
        if not all(member in top_10_dict.get(other, []) for other in potential_group if other != member):
            potential_group.discard(member)

    # 确保每个组至少有两个成员
    if len(potential_group) >= 2 and potential_group not in groups:
        groups.append(potential_group)

# 输出找到的组
for group in groups:
    print(group)

# 转换时间数据
df = pd.read_csv("/content/drive/My Drive/112 Sesmester/data2/fi5.csv")
df = convert_time(df, time_cols)

{2, 34428}
{45, 7468, 32101}
{3958, 727}
{16448, 28700, 1910}
{35632, 12816, 3000, 24847}
{15269, 3159, 11261, 15134}
{9486, 31260}
{26819, 26907, 4787, 30566}
{5395, 18116}
{3159, 15134}
{7080, 36148, 17913}
{18417, 31602, 7406}
{7080, 36148}
{9018, 14815}
{9095, 11261, 15134}
{20578, 9731, 26453, 26649}
{10789, 9822}
{36148, 17913, 39484}
{10376, 23138, 14815}
{10789, 27183}
{35611, 10907}
{3159, 11261, 15134}
{36148, 7477}
{9486, 13940}
{7080, 17832, 31575, 24408}
{16341, 3159, 11261}
{19304, 16615}
{16341, 11261}
{9095, 15134}
{17928, 38281, 24014}
{37600, 5395, 18116}
{28198, 18348, 9486, 31260}
{18417, 40244}
{28700, 18909}
{19017, 22922, 20867}
{29570, 32994, 24014, 19130}
{17314, 19981, 31955}
{20578, 9731, 21066}
{22250, 24436, 28540}
{22818, 45}
{24014, 19130}
{28577, 22250, 24436, 28540}
{36148, 39484}
{32810, 26453, 26649}
{26819, 26907}
{4787, 26907}
{10311, 17913, 36148, 39484}
{27776, 28540}
{28700, 1910}
{31602, 7406}
{32101, 19130}
{20578, 9731, 32810, 26453, 26649}
{3

Converting Time: 100%|██████████| 1/1 [00:00<00:00, 51.89it/s]


In [ ]:
distance_cache={}

# 初始化用于存储每个数据点最佳组的字典
best_group_for_points = {}

# 初始化用于存储每组的碳减排量
group_carbon_reductions = {}

# 遍历每个组
for group in groups:
    group_list = list(group)
    group_subset = dfo.loc[group_list]
    total_distance_km = group_subset['Distance_km'].sum()
    min_distance = process_dataframe(group_subset, distance_cache)[1]
    carbon_reduction_g = (total_distance_km - min_distance) * 50.8

    # 如果 carbon_reduction_g 为负，则跳过当前组
    if carbon_reduction_g < 0:
        continue

    # 存储每个组的碳减排量
    group_carbon_reductions[tuple(group_list)] = carbon_reduction_g

# 为每个点找到最佳组
for point in dfo.index:
    best_group = None
    best_carbon_reduction = -float('inf')

    for group, carbon_reduction in group_carbon_reductions.items():
        if point in group and carbon_reduction > best_carbon_reduction:
            best_carbon_reduction = carbon_reduction
            best_group = group

    if best_group is not None:
        best_group_for_points[point] = best_group
# 根据最佳组创建最终结果

In [ ]:
# 初始化用于存储每个数据点最佳组的字典
best_group_for_points

{45: (22818, 45),
 1910: (28700, 1910),
 3000: (35632, 12816, 3000, 24847),
 3159: (15269, 3159, 11261, 15134),
 7080: (7080, 17832, 31575, 24408),
 7406: (31602, 7406),
 7477: (36148, 7477),
 9018: (9018, 14815),
 9486: (28198, 18348, 9486, 31260),
 9731: (20578, 9731, 26453, 26649),
 10311: (10311, 17913, 36148, 39484),
 10907: (35611, 10907),
 11261: (15269, 3159, 11261, 15134),
 12816: (35632, 12816, 3000, 24847),
 13940: (9486, 13940),
 14815: (9018, 14815),
 15134: (15269, 3159, 11261, 15134),
 15269: (15269, 3159, 11261, 15134),
 16341: (16341, 3159, 11261),
 17832: (7080, 17832, 31575, 24408),
 17913: (10311, 17913, 36148, 39484),
 18348: (28198, 18348, 9486, 31260),
 18417: (18417, 31602, 7406),
 19130: (32101, 19130),
 20578: (20578, 9731, 26453, 26649),
 22818: (22818, 45),
 24408: (7080, 17832, 31575, 24408),
 24847: (35632, 12816, 3000, 24847),
 26453: (20578, 9731, 26453, 26649),
 26649: (20578, 9731, 26453, 26649),
 27776: (27776, 28540),
 28198: (28198, 18348, 9486, 312

In [ ]:
final_results = []

for point, best_group in best_group_for_points.items():
    group_subset = dfo.loc[list(best_group)]
    max_time_diff_minutes = group_subset['Time_Orderd_in_minutes'].max() - group_subset['Time_Orderd_in_minutes'].min()
    carbon_reduction_g = group_carbon_reductions[best_group]

    # 对 group_subset 的每一行进行迭代
    for idx, row in group_subset.iterrows():
        # 将行转换为字典，并添加额外的信息
        row_dict = row.to_dict()
        row_dict.update({
            'Group': best_group,
            'Idx': idx,
            'Restaurant_Lat': row['Restaurant_latitude'],
            'Restaurant_Lon': row['Restaurant_longitude'],
            'Delivery_Lat': row['Delivery_location_latitude'],
            'Delivery_Lon': row['Delivery_location_longitude'],
            'Total_Distance_km': total_distance_km,
            'Max_Time_Diff_Minutes': max_time_diff_minutes,
            'Shortest_Path_Distance_km': min_distance,
            'Carbon_Reduction_g': carbon_reduction_g,
            'Group_Length': len(group_subset)
        })

        # 添加到结果列表
        final_results.append(row_dict)


# 创建最终 DataFrame
final_results_df = pd.DataFrame(final_results)
final_results_df

""


In [ ]:
import itertools
import requests


location_cols = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']
def get_route_distance(lat1, lon1, lat2, lon2, distance_cache):
    # 构建请求的键
    cache_key = (lat1, lon1, lat2, lon2)

    # 检查缓存中是否已有该路径的距离
    if cache_key in distance_cache:
        return distance_cache[cache_key]

    # 如果缓存中没有，发起新的 API 请求
    url = f"http://router.project-osrm.org/route/v1/car/{lon1},{lat1};{lon2},{lat2}?overview=false"
    response = requests.get(url)
    data = response.json()

    if 'routes' in data and len(data['routes']) > 0:
        distance = data['routes'][0]['distance'] / 1000
        # 存储距离到缓存
        distance_cache[cache_key] = distance
        return distance
    else:
        return None

def process_dataframe(df, distance_cache):
    # 提取每行的餐厅和送餐点位置
    points = df[location_cols].values.tolist()

    # 分别提取所有餐厅和所有送餐点的坐标
    restaurants = [row[:2] for row in points]
    deliveries = [row[2:] for row in points]

    # 初始化最短路径和最小距离
    shortest_path = None
    min_distance = float('inf')

    # 使用 tqdm 来显示进度条
    for rest_order in tqdm(itertools.permutations(restaurants), desc="Calculating restaurant paths"):
        for del_order in tqdm(itertools.permutations(deliveries), desc="Calculating delivery paths"):
            total_distance = 0
            # 计算所有餐厅之间的距离
            for i in range(len(rest_order) - 1):
                distance = get_route_distance(rest_order[i][0], rest_order[i][1], rest_order[i + 1][0], rest_order[i + 1][1],distance_cache)
                if distance is None:
                    total_distance = float('inf')
                    break
                total_distance += distance

            # 计算最后一个餐厅到第一个送餐点的距离
            if total_distance != float('inf'):
                distance = get_route_distance(rest_order[-1][0], rest_order[-1][1], del_order[0][0], del_order[0][1],distance_cache)
                if distance is None:
                    continue
                total_distance += distance

            # 计算所有送餐点之间的距离
            for i in range(len(del_order) - 1):
                distance = get_route_distance(del_order[i][0], del_order[i][1], del_order[i + 1][0], del_order[i + 1][1],distance_cache)
                if distance is None:
                    total_distance = float('inf')
                    break
                total_distance += distance

            if total_distance < min_distance:
                min_distance = total_distance
                shortest_path = list(rest_order) + list(del_order)

    return shortest_path, min_distance

distance_cache = {}

for group in groups:

  print(group)
  group_subset = df.loc[group]  # 从原始 df 中提取某个组的子集
  # print(group_subset)
  total_distance_km = group_subset['Distance_km'].sum()
  max_time_diff_minutes = group_subset['Time_Orderd_in_minutes'].max() - group_subset['Time_Orderd_in_minutes'].min()
  shortest_path, min_distance = process_dataframe(group_subset, distance_cache)
  print(f"Shortest Path: {shortest_path}, Distance: {min_distance} km ,減碳量: {(total_distance_km-min_distance)*50.8}g, 時間差: {max_time_diff_minutes}min")


<ipython-input-17-dee0054c9025>:77: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  group_subset = df.loc[group]  # 从原始 df 中提取某个组的子集


{34301, 875}


Calculating restaurant paths: 0it [00:00, ?it/s]
Calculating delivery paths: 0it [00:00, ?it/s]
Calculating delivery paths: 1it [00:01,  1.44s/it]
Calculating delivery paths: 2it [00:02,  1.22s/it]
Calculating restaurant paths: 1it [00:02,  2.46s/it]
Calculating delivery paths: 0it [00:00, ?it/s]
Calculating delivery paths: 1it [00:00,  1.02it/s]
Calculating delivery paths: 2it [00:01,  1.35it/s]
Calculating restaurant paths: 2it [00:03,  1.98s/it]
<ipython-input-17-dee0054c9025>:77: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  group_subset = df.loc[group]  # 从原始 df 中提取某个组的子集


Shortest Path: [[22.538731, 88.364878], [22.552672, 88.352885], [22.622672, 88.422885], [22.618731, 88.444878]], Distance: 20.1541 km ,減碳量: 505.3838000000001g, 時間差: 20min
{39089, 3035}


Calculating restaurant paths: 0it [00:00, ?it/s]
Calculating delivery paths: 0it [00:00, ?it/s]
Calculating delivery paths: 1it [00:01,  1.48s/it]
Calculating delivery paths: 2it [00:02,  1.24s/it]
Calculating restaurant paths: 1it [00:02,  2.49s/it]
Calculating delivery paths: 0it [00:00, ?it/s]
Calculating delivery paths: 1it [00:00,  1.02it/s]
Calculating delivery paths: 2it [00:01,  1.34it/s]
Calculating restaurant paths: 2it [00:03,  1.99s/it]
<ipython-input-17-dee0054c9025>:77: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  group_subset = df.loc[group]  # 从原始 df 中提取某个组的子集


...

Shortest Path: [[22.551084, 88.354127], [22.552672, 88.352885], [22.621084, 88.424127], [22.632672, 88.432885]], Distance: 17.18 km ,減碳量: 625.9525199999999g, 時間差: 10min
{23634, 6302}


Calculating restaurant paths: 0it [00:00, ?it/s]
Calculating delivery paths: 0it [00:00, ?it/s]
Calculating delivery paths: 1it [00:01,  1.43s/it]
Calculating delivery paths: 2it [00:01,  1.03it/s]
Calculating restaurant paths: 1it [00:01,  1.94s/it]
Calculating delivery paths: 0it [00:00, ?it/s]
Calculating delivery paths: 1it [00:00,  1.01it/s]
Calculating delivery paths: 2it [00:01,  1.34it/s]
Calculating restaurant paths: 2it [00:03,  1.72s/it]
<ipython-input-17-dee0054c9025>:77: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  group_subset = df.loc[group]  # 从原始 df 中提取某个组的子集


Shortest Path: [[22.538999, 88.322337], [22.552996, 88.35231], [22.608999, 88.392336], [22.662996, 88.46231]], Distance: 28.1702 km ,減碳量: 391.1396799999998g, 時間差: 5min
{35788, 38489}


Calculating restaurant paths: 0it [00:00, ?it/s]
Calculating delivery paths: 2it [00:00, 13911.46it/s]

Calculating delivery paths: 2it [00:00, 14388.69it/s]
Calculating restaurant paths: 2it [00:00, 90.01it/s]

Shortest Path: [[22.577821, 88.400581], [22.569358, 88.433452], [22.667821, 88.490581], [22.649358, 88.513452]], Distance: 31.4179 km ,減碳量: 500.30888000000004g, 時間差: 0min


In [ ]:
import folium

import requests
import polyline

def get_route(points):
    # 将点列表转换为OSRM API所需的格式
    coordinates = ';'.join([f"{lon},{lat}" for lat, lon in points])

    # 构建请求 URL
    url = f"http://router.project-osrm.org/route/v1/driving/{coordinates}?overview=full"

    # 发送请求
    response = requests.get(url)

    # 解析 JSON 数据
    data = response.json()

    # 检查是否有 'routes' 键
    if 'routes' in data and len(data['routes']) > 0:
        # 解码 Polyline
        route_polyline = data['routes'][0]['geometry']
        route = polyline.decode(route_polyline)

        # 转换坐标格式
        route = [[lat, lon] for lat, lon in route]

        return route
    else:
        # 未找到有效路径时返回 None 或适当的错误处理
        return None

m = folium.Map(location=[22.5726, 88.3639], zoom_start=12)

# 创建一个地图对象



# 绘制 group_subset['Route'] 的路线
for idx, row in group_subset.iterrows():
    route = eval(row['Route'])  # 假设 Route 是一个字符串表示的列表
    folium.PolyLine(route, color='blue', weight=2.5, opacity=1).add_to(m)
    folium.Marker(route[0], popup=f"Start of Route {idx}").add_to(m)
    folium.Marker(route[-1], popup=f"End of Route {idx}").add_to(m)
route = get_route(shortest_path)
folium.PolyLine(route, color='green', weight=2.5, opacity=1).add_to(m)
# 显示地图
# m.save('path_map.html')
m

In [ ]:
group
group_subset = df.loc[group]
points = group_subset[location_cols].values.tolist()
paths = [(row[:2], row[2:]) for row in points]
# len(paths)
paths

# 为每对坐标重命名为 Ra, Da, Rb, Db, ...
renamed_paths = [(f"R{i+1}", f"D{i+1}") for i, _ in enumerate(paths)]

# 输出重命名后的 paths
for i, (restaurant, delivery) in enumerate(paths):
    print(f"{renamed_paths[i][0]}: {restaurant}, {renamed_paths[i][1]}: {delivery}")

R1: [22.514119, 88.362504], D1: [22.604119, 88.452504]
R2: [22.539129, 88.365507], D2: [22.669129, 88.495507]
R3: [22.527893, 88.368628], D3: [22.637893, 88.478628]
R4: [22.533662, 88.366217], D4: [22.613662, 88.446217]
R5: [22.551084, 88.354127], D5: [22.661084, 88.464127]


<ipython-input-55-df0b31aa0605>:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  group_subset = df.loc[group]


In [ ]:
all_possible_paths = [tuple(row[i:i+2] for i in range(0, len(row), 2)) for row in points]
len(all_possible_paths)
# 筛选出满足条件的路径（送餐点在餐厅后面）
valid_paths = [path for path in all_possible_paths if all(path[i][1] != path[i+1][0] for i in range(len(path)-1))]
len(valid_paths)
all_possible_paths

[([22.514119, 88.362504], [22.604119, 88.452504]),
 ([22.539129, 88.365507], [22.669129, 88.495507]),
 ([22.527893, 88.368628], [22.637893, 88.478628]),
 ([22.533662, 88.366217], [22.613662, 88.446217]),
 ([22.551084, 88.354127], [22.661084, 88.464127])]